## This notebook was created in Google Colab using an A100 GPU as it is very resource intensive

In [ ]:
!pip install tensorflow

In [ ]:
# install keras_tunes
!pip install keras-tuner --upgrade

In [3]:
#Dependencies
#building a model
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
#tuner
import keras_tuner
#splitting data
from sklearn.model_selection import train_test_split

#### Mounting google drive in order to read images from kaggle API

In [4]:
#Mount Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#upload kaggle.json
#instructions for accessing kaggle api can be found here: https://www.kaggle.com/discussions/general/156610
from google.colab import files

files.upload()

In [ ]:
#import images from kaggle
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d unmoved/30k-cats-and-dogs-150x150-greyscale
!unzip 30k-cats-and-dogs-150x150-greyscale.zip

### Using ImageDataGenerator to combine cats and dogs folders into one dataset with labels 0:Cat, 1:Dog

In [7]:
# Create an ImageDataGenerator object
train_datagen = ImageDataGenerator(rescale = 1./255)

# Create a flow of images from the folders and set their class labels
train_generator = train_datagen.flow_from_directory(
    directory = 'Animal Images',
    target_size = (150, 150),
    color_mode = 'grayscale',
    class_mode = 'binary',
    batch_size = 30000
)

Found 30061 images belonging to 2 classes.


In [8]:
#split data into training and test sets
X, y = train_generator.next()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [9]:
print("X_train shape:",X_train.shape)
print("X_val shape:",X_test.shape)
print("Y_train shape:",y_train.shape)
print("Y_val shape:",y_test.shape)

X_train shape: (27000, 150, 150, 1)
X_val shape: (3000, 150, 150, 1)
Y_train shape: (27000,)
Y_val shape: (3000,)


### Experimenting and building our CNN model

In [10]:
#build base model
base_model=Sequential()
base_model.add(Convolution2D(16,3,3,input_shape=(150,150,1),activation='relu'))
base_model.add(MaxPooling2D(pool_size=(2,2)))
base_model.add(Flatten())
base_model.add(Dense(128,activation='relu'))
base_model.add(Dense(1,activation='sigmoid'))
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 50, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 16)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 10000)             0         
                                                                 
 dense (Dense)               (None, 128)               1280128   
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,417
Trainable params: 1,280,417
Non-trainable params: 0
______________________________________________

In [11]:
# compile model
optimizer = Adam(learning_rate=0.001)
base_model.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics=["accuracy"])

In [12]:
# train model
base_model.fit(X_train,y_train, epochs=25,validation_data=(X_test,y_test))

Epoch 1/25
844/844 [==============================] - 15s 5ms/step - loss: 0.6303 - accuracy: 0.6455 - val_loss: 0.5922 - val_accuracy: 0.6857
Epoch 2/25
844/844 [==============================] - 3s 4ms/step - loss: 0.5457 - accuracy: 0.7249 - val_loss: 0.5589 - val_accuracy: 0.7097
Epoch 3/25
844/844 [==============================] - 3s 4ms/step - loss: 0.4995 - accuracy: 0.7551 - val_loss: 0.5547 - val_accuracy: 0.7193
Epoch 4/25
844/844 [==============================] - 3s 4ms/step - loss: 0.4550 - accuracy: 0.7840 - val_loss: 0.6022 - val_accuracy: 0.6973
Epoch 5/25
844/844 [==============================] - 3s 4ms/step - loss: 0.4021 - accuracy: 0.8163 - val_loss: 0.5668 - val_accuracy: 0.7283
Epoch 6/25
844/844 [==============================] - 3s 4ms/step - loss: 0.3407 - accuracy: 0.8511 - val_loss: 0.5783 - val_accuracy: 0.7330
Epoch 7/25
844/844 [==============================] - 3s 4ms/step - loss: 0.2714 - accuracy: 0.8896 - val_loss: 0.6650 - val_accuracy: 0.7293
Epoch

In [14]:
# print model loss and accuracy on test set
model_loss, model_accuracy = base_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

94/94 - 0s - loss: 1.8494 - accuracy: 0.7220 - 248ms/epoch - 3ms/step
Loss: 1.8494117259979248, Accuracy: 0.722000002861023


In [15]:
# model using different activation and more convolution/pooling layers
# added dropout layer
model2 = Sequential()

model2.add(Convolution2D(12, (3,3),activation ='leaky_relu', input_shape = X_train.shape[1:]))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Convolution2D(24, (3,3),activation ='leaky_relu'))
model2.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))


model2.add(Flatten())
model2.add(Dense(128, activation = "leaky_relu"))
# add dropout layer
model2.add(Dropout(0.1))
model2.add(Dense(1, activation = "sigmoid"))

In [16]:
# compile model
#different learning rates tested 0.1, 0.01, 0.001, 0.0001
optimizer = Adam(learning_rate=0.001)
model2.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics=["accuracy"])

In [17]:
# train model
# different batch_sizes were tested: 100,200,250,300
batch_size=200
model2.fit(X_train,y_train,batch_size = batch_size, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
135/135 [==============================] - 5s 18ms/step - loss: 0.6927 - accuracy: 0.5865 - val_loss: 0.6326 - val_accuracy: 0.6473
Epoch 2/20
135/135 [==============================] - 2s 14ms/step - loss: 0.5690 - accuracy: 0.7074 - val_loss: 0.5497 - val_accuracy: 0.7213
Epoch 3/20
135/135 [==============================] - 2s 14ms/step - loss: 0.4992 - accuracy: 0.7576 - val_loss: 0.5305 - val_accuracy: 0.7403
Epoch 4/20
135/135 [==============================] - 2s 14ms/step - loss: 0.4471 - accuracy: 0.7884 - val_loss: 0.5218 - val_accuracy: 0.7390
Epoch 5/20
135/135 [==============================] - 2s 14ms/step - loss: 0.4006 - accuracy: 0.8166 - val_loss: 0.5538 - val_accuracy: 0.7310
Epoch 6/20
135/135 [==============================] - 2s 14ms/step - loss: 0.3456 - accuracy: 0.8474 - val_loss: 0.5698 - val_accuracy: 0.7433
Epoch 7/20
135/135 [==============================] - 2s 15ms/step - loss: 0.2876 - accuracy: 0.8785 - val_loss: 0.5869 - val_accuracy: 0.7370

In [18]:
# print model loss and accuracy
model_loss, model_accuracy = model2.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


94/94 - 0s - loss: 1.4796 - accuracy: 0.7280 - 460ms/epoch - 5ms/step
Loss: 1.4795571565628052, Accuracy: 0.7279999852180481


### Using Keras-tuner to run trials with different hyperparameters/layers

In [19]:
# params from https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range= 0.3,
        zoom_range= 0.3,
        width_shift_range= 0,
        height_shift_range= 0,
        horizontal_flip=False,
        vertical_flip=False)
datagen.fit(X_train)

In [20]:
# function for keras_tuner
def tuner(hp):
  #set kernel_size and pool_size vairables
  kernel_size = (3,3)
  pool_size = (2,2)
  #set ranges for dropout coefficients
  dropout_coeff_conv = hp.Float("conv_dropout", min_value=0.05, max_value=0.2)
  dropout_coeff_hidden = hp.Float("hidden_dropout", min_value=0.05, max_value=0.2)
  #set ranges for filters, convolutional layers, hidden layers, hidden layer units
  filter_num = hp.Int('filter_num', min_value=2, max_value=24, step=2)
  conv_layer_num =hp.Int('conv_layer', min_value=1, max_value=5, step=1)
  hidden_layer_num = hp.Int('hidden_layer_num', min_value=1, max_value=5, step=1)
  layer_unit = hp.Int('layer_unit',156,1048,100)
  #output and input shapes
  output_num = y_train.shape[0]
  input_shape = X_train.shape[1:]
  #best learning rate from model2
  learning_rate = 0.001

  #build model
  model = Sequential()
  #test different convolutional layers
  for i in range(conv_layer_num):
      model.add(Convolution2D(
          filters=filter_num, kernel_size=kernel_size, padding="same"
          ,activation='relu', input_shape = input_shape))
      filter_num*=2
      model.add(MaxPooling2D(pool_size = pool_size,padding="same"))
      model.add(Dropout(dropout_coeff_conv))

  # test different fully connected layers
  model.add(Flatten())

  print("layer unit(hidden):",layer_unit)
  for i in range(hidden_layer_num):
      model.add(Dense(layer_unit, activation= 'relu'))
      model.add(Dropout(dropout_coeff_hidden))
  model.add(Dense(1,activation='sigmoid'))

  #compile the model
  optimizer = Adam(learning_rate=learning_rate)
  model.compile(optimizer=optimizer,loss = "binary_crossentropy",
              metrics=["accuracy"])
  return model

 #Reduce learning rate when a metric has stopped improving.
 # source: https://keras.io/api/callbacks/reduce_lr_on_plateau/
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)

#Number of trials to run
trial_number = 10

#target validation accuracy
tuner = keras_tuner.RandomSearch(
    tuner,
    objective='val_accuracy',
    max_trials = trial_number,
    overwrite=True,
    project_name="cats_or_dogs_CNN")

tuner.search(datagen.flow(X_train,y_train, batch_size=64).x, datagen.flow(X_train,y_train, batch_size=64).y,
             epochs=10,batch_size = 250, validation_data=(X_test, y_test),callbacks=[reduce_lr])
best_model = tuner.get_best_models()[0]

Trial 10 Complete [00h 00m 48s]
val_accuracy: 0.8396666646003723

Best val_accuracy So Far: 0.8396666646003723
Total elapsed time: 00h 05m 56s
layer unit(hidden): 556


In [21]:
# Best model from keras_tuner search
# this model summary is from second tuner run and
#isn't as good as first tuner run
# see catDog_model below for best overall model

best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 22)      220       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 22)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 75, 75, 22)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 44)        8756      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 38, 44)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 38, 38, 44)        0

### Save models to load into Model_Test.ipynb

In [ ]:
#save model to current session and google drive
# FIRST TUNER RUN

# best_model.save('cat_dog_CNN.h5')
# best_model.save('/content/gdrive/MyDrive/cat_or_dog_CNN.keras')

In [ ]:
#save model to current session and google drive
# SECOND TUNER RUN

# best_model.save('cat_dog_CNN_2.h5')
# best_model.save('/content/gdrive/MyDrive/cat_or_dog_CNN_2.keras')

In [22]:
# print model loss and accuracy for best model of 2nd tuner run
model_loss, model_accuracy = best_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

94/94 - 1s - loss: 0.3537 - accuracy: 0.8397 - 927ms/epoch - 10ms/step
Loss: 0.35369279980659485, Accuracy: 0.8396666646003723


In [23]:
# load in best model from first tuner run
catDog_model = keras.models.load_model('/content/gdrive/MyDrive/cat_or_dog_CNN.keras')

In [24]:
# print model loss and accuracy
# first tuner run is best with 90.87% accuracy on test set
model_loss, model_accuracy = catDog_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

94/94 - 1s - loss: 0.2579 - accuracy: 0.8957 - 678ms/epoch - 7ms/step
Loss: 0.25789788365364075, Accuracy: 0.8956666588783264


In [25]:
catDog_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 8)       80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 8)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 75, 75, 8)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 16)        1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 38, 16)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 38, 38, 16)        0

In [26]:
# print model loss and accuracy
# first tuner run is best with 90.87% accuracy on test set
model_loss, model_accuracy = catDog_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

94/94 - 0s - loss: 0.2579 - accuracy: 0.8957 - 300ms/epoch - 3ms/step
Loss: 0.25789788365364075, Accuracy: 0.8956666588783264
